<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LFM2_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from openai import OpenAI

# --- API Key Retrieval (CRITICAL: Replace placeholder or set environment variable) ---
try:
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY not found in Colab secrets.")
except Exception:
    # IMPORTANT: REPLACE THIS WITH YOUR ACTUAL OPENROUTER API KEY
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "YOUR_OPENROUTER_API_KEY_PLACEHOLDER")



# 1. SET UP THE OPENROUTER CLIENT
# OpenRouter uses the standard OpenAI client, but points the base_url to OpenRouter's endpoint.
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    # Optional but recommended by OpenRouter for ranking/tracking:
    default_headers={"HTTP-Referer": "YourLFM2Demo", "X-Title": "LFM2 OpenRouter Demo"}
)

# 2. CHOOSE THE LFM2 MODEL
LFM2_MODEL_ID = "liquid/lfm-2.2-6b"
LFM2_MODEL_ID = "liquid/lfm2-8b-a1b"

# 3. DEFINE THE CONVERSATION PROMPT
messages = [
    {"role": "user", "content": "Explain the core difference between edge AI and cloud AI in simple terms."}
]

print(f"--- Sending request to Model: {LFM2_MODEL_ID} ---")
print(f"Prompt: {messages[0]['content']}\n")

# 4. GET THE COMPLETION (DEMO)
try:
    response = client.chat.completions.create(
        model=LFM2_MODEL_ID,
        messages=messages,
        temperature=0.7 # Add a temperature setting for creativity
    )

    # 5. PRINT THE MODEL'S RESPONSE
    lfm2_response = response.choices[0].message.content
    print("--- LFM2 Response ---")
    print(lfm2_response.strip())

    # OPTIONAL: Print cost details
    print("\n--- Usage Details ---")
    # OpenRouter includes usage details in the response
    usage = response.usage
    print(f"Tokens Used: Input={usage.prompt_tokens}, Output={usage.completion_tokens}, Total={usage.total_tokens}")

except Exception as e:
    print(f"An error occurred: {e}")

--- Sending request to Model: liquid/lfm2-8b-a1b ---
Prompt: Explain the core difference between edge AI and cloud AI in simple terms.

--- LFM2 Response ---
The core difference between edge AI and cloud AI lies in where the data is processed:

- **Cloud AI** processes data on remote servers over the internet. It’s powerful and flexible, but requires sending data to the cloud, which can introduce delays and needs a stable connection.

- **Edge AI** processes data right on the device where it’s generated—like a smartphone, smartwatch, or self-driving car—locally, without sending data far away. This means faster responses and better privacy, since sensitive data stays on the device.

In short: Cloud AI relies on distant servers for powerful processing, while edge AI uses local devices for faster, more private, and immediate decisions.

--- Usage Details ---
Tokens Used: Input=24, Output=142, Total=166


## AGENTIC - SHORT - HAUL

In [8]:
import os
import json
import warnings # <-- ADDED: For warning suppression
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional

# Suppress all deprecation warnings from Pydantic V1 -> V2
warnings.filterwarnings('ignore', category=DeprecationWarning) # <-- ADDED: Warning suppression

# --- 1. CONFIGURATION ---
# --- API Key Retrieval (CRITICAL: Replace placeholder or set environment variable) ---
try:
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY not found in Colab secrets.")
except Exception:
    # IMPORTANT: YOU MUST REPLACE THIS WITH YOUR ACTUAL OPENROUTER API KEY
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "YOUR_OPENROUTER_API_KEY_PLACEHOLDER")


LFM2_MODEL_ID = "liquid/lfm2-8b-a1b"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

# --- 2. STRUCTURED OUTPUT SCHEMA (PYDANTIC) ---
class RouteSegment(BaseModel):
    """A segment of the flight route."""
    waypoint: str = Field(description="The ICAO identifier for the waypoint, VOR, or intersection.")
    type: str = Field(description="Type of waypoint (e.g., 'DEP_APT', 'VOR', 'JET_WAY', 'ARR_APT').")
    altitude_ft: int = Field(description="The cruising altitude in feet for this segment.")

class FlightPlanOutput(BaseModel):
    """The final structured flight plan generated by the EdgePilot agent."""
    flight_number: str = Field(description="The flight identifier (e.g., 'AAL123').")
    departure_icao: str = Field(description="ICAO code for the departure airport (e.g., 'KLAX').")
    arrival_icao: str = Field(description="ICAO code for the arrival airport (e.g., 'KJFK').")
    estimated_time_en_route_hrs: float = Field(description="Total estimated flight time in hours.")
    fuel_required_kg: int = Field(description="Total calculated fuel required, including reserve, in kilograms.")
    route: List[RouteSegment] = Field(description="The sequence of waypoints defining the flight route.")
    risk_assessment: str = Field(description="Concise summary of the primary risk (e.g., 'Low-Level Wind Shear at Destination').")
    pilot_notes: str = Field(description="Short, critical instructions for the pilot (e.g., 'Check NOTAMs for R-400 restriction active from 1400Z').")


# --- 3. AGENT TOOLS (SIMULATED EXTERNAL APIS) ---
def get_realtime_weather(airport_icao: str) -> str:
    weather_data = {
        "KLAX": "METAR KLAX 051853Z 25010KT 10SM SCT040 BKN070 15/07 A3001 RMK A02",
        "KJFK": "TAF KJFK 051730Z 0518/0618 20008KT P6SM BKN030 OVC080 TEMPO 0600/0604 4SM SHRA BKN015",
    }
    return weather_data.get(airport_icao.upper(), f"No real-time data found for {airport_icao}.")

def get_notam_data(region_icao: str) -> str:
    notam_data = {
        "WEST_US": "NOTAM 05/023: Runway 25L at KLAX CLOSED for maintenance until 061200Z. Check current ATIS.",
        "EAST_US": "NOTAM 05/045: Restricted Airspace R-400 active for military exercises 1400Z-1800Z. Alternate routing required.",
    }
    return notam_data.get(region_icao.upper(), f"No active NOTAMs of note for {region_icao}.")


# --- 4. AGENT EXECUTION (SINGLE-TURN LOGIC) ---

def run_edge_pilot_agent(
    initial_user_prompt: str,
    model_id: str = LFM2_MODEL_ID
) -> dict:

    if "YOUR_OPENROUTER_API_KEY_PLACEHOLDER" in OPENROUTER_API_KEY:
        return {"status": "error", "message": "API Key placeholder detected. Please replace 'YOUR_OPENROUTER_API_KEY_PLACEHOLDER' with your actual key."}

    client = OpenAI(
        base_url=OPENROUTER_BASE_URL,
        api_key=OPENROUTER_API_KEY,
        default_headers={"HTTP-Referer": "EdgePilotAgent-SingleTurn", "X-Title": "LFM2 Flight Planner"}
    )

    # --- STEP 1: EXECUTE LOCAL TOOLS AND GATHER DATA ---
    print("[AGENT STEP 1] Pre-fetching required external data (local simulation)...")
    weather_klax = get_realtime_weather("KLAX")
    weather_kjfk = get_realtime_weather("KJFK")
    notams_west = get_notam_data("WEST_US")
    notams_east = get_notam_data("EAST_US")

    context_data = f"""
    --- REAL-TIME FLIGHT CONTEXT DATA ---
    1. KLAX Weather (Departure): {weather_klax}
    2. KJFK Weather (Arrival): {weather_kjfk}
    3. West US NOTAMs: {notams_west}
    4. East US NOTAMs: {notams_east}
    --- END CONTEXT ---
    """

    # --- STEP 2: BUILD THE SINGLE PROMPT ---
    messages = [
        {
            "role": "system",
            "content": (
                "You are EdgePilot, an expert Aviation Dispatch Agent. "
                "Your task is to generate the SAFEST and most OPTIMIZED flight plan based on the user's request AND the provided CONTEXT DATA. "
                "The final response MUST be a single JSON object that strictly conforms to the FlightPlanOutput Pydantic schema. "
                "Do not include any explanatory text or markdown formatting outside the single JSON block."
                # FIX: Using V2 Pydantic method
                f"SCHEMA: {json.dumps(FlightPlanOutput.model_json_schema())}"
            )
        },
        {"role": "user", "content": f"{initial_user_prompt}\n\n{context_data}"}
    ]

    # --- STEP 3: CALL LFM2 ---
    try:
        print("[AGENT STEP 2] Calling LFM2 Model for final plan generation (Single-Turn)...")
        response = client.chat.completions.create(
            model=model_id,
            messages=messages,
            temperature=0.0,
        )

        # --- STEP 4: PARSE AND VALIDATE OUTPUT ---
        raw_json = response.choices[0].message.content
        print("[AGENT SUCCESS] Final response received. Parsing JSON...")

        # FIX: Using V2 Pydantic methods
        return FlightPlanOutput.model_validate_json(raw_json).model_dump()

    except Exception as e:
        error_message = str(e)
        print(f"\n--- ERROR ---: {error_message}")
        # ... (Error handling remains the same) ...
        return {"status": "error", "message": f"An unexpected error occurred (Check JSON format): {error_message}"}


# --- 5. EXECUTION ---
if __name__ == "__main__":

    request = "Plan a flight from Los Angeles (KLAX) to New York (KJFK) for an A320, optimizing for the shortest possible route but factoring in all current NOTAMs and weather."

    print(f"REQUEST: {request}\n")

    final_plan = run_edge_pilot_agent(request)

    print("\n" + "="*50)
    print("✈️ FINAL LFM2 EDGE PILOT FLIGHT PLAN RESULT ✈️")
    print("="*50)
    print(json.dumps(final_plan, indent=4))
    print("\nAgentic Loop Complete.")

REQUEST: Plan a flight from Los Angeles (KLAX) to New York (KJFK) for an A320, optimizing for the shortest possible route but factoring in all current NOTAMs and weather.

[AGENT STEP 1] Pre-fetching required external data (local simulation)...
[AGENT STEP 2] Calling LFM2 Model for final plan generation (Single-Turn)...
[AGENT SUCCESS] Final response received. Parsing JSON...

✈️ FINAL LFM2 EDGE PILOT FLIGHT PLAN RESULT ✈️
{
    "flight_number": "A320-789",
    "departure_icao": "KLAX",
    "arrival_icao": "KJFK",
    "estimated_time_en_route_hrs": 6.75,
    "fuel_required_kg": 28500,
    "route": [
        {
            "waypoint": "KLAX",
            "type": "DEP_APT",
            "altitude_ft": 25000
        },
        {
            "waypoint": "DEN",
            "type": "JET_WAY",
            "altitude_ft": 35000
        },
        {
            "waypoint": "ORD",
            "type": "JET_WAY",
            "altitude_ft": 35000
        },
        {
            "waypoint": "KJFK",
  

## AGENTIC - LONG - HAUL

In [10]:
import os
import json
import warnings
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional

# Suppress Pydantic deprecation warnings that occur in some environments
warnings.filterwarnings('ignore', category=DeprecationWarning)

# --- 1. CONFIGURATION ---
# --- API Key Retrieval (CRITICAL: Replace placeholder or set environment variable) ---
try:
    # Colab environment variable logic
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY not found in Colab secrets.")
except Exception:
    # IMPORTANT: YOU MUST REPLACE THIS WITH YOUR ACTUAL OPENROUTER API KEY
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "YOUR_OPENROUTER_API_KEY_PLACEHOLDER")


LFM2_MODEL_ID = "liquid/lfm2-8b-a1b" # The high-reasoning MoE model
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

# --- 2. STRUCTURED OUTPUT SCHEMA (PYDANTIC) ---
class RouteSegment(BaseModel):
    """A segment of the flight route."""
    waypoint: str = Field(description="The ICAO identifier for the waypoint, VOR, or intersection.")
    type: str = Field(description="Type of waypoint (e.g., 'DEP_APT', 'VOR', 'JET_WAY', 'OCEANIC').")
    altitude_ft: int = Field(description="The cruising altitude in feet for this segment.")

class FlightPlanOutput(BaseModel):
    """The final structured flight plan generated by the EdgePilot agent."""
    flight_number: str = Field(description="The flight identifier (e.g., 'B77W-110').")
    departure_icao: str = Field(description="ICAO code for the departure airport (e.g., 'KJFK').")
    arrival_icao: str = Field(description="ICAO code for the arrival airport (e.g., 'ZSPD').")
    estimated_time_en_route_hrs: float = Field(description="Total estimated flight time in hours.")
    fuel_required_kg: int = Field(description="Total calculated fuel required, including reserve, in kilograms.")
    route: List[RouteSegment] = Field(description="The sequence of waypoints defining the flight route.")
    risk_assessment: str = Field(description="Concise summary of the primary risk.")
    pilot_notes: str = Field(description="Short, critical instructions for the pilot.")


# --- 3. AGENT TOOLS (SIMULATED EXTERNAL APIS) ---
# NOTE: Mock data updated for KJFK-ZSPD route and constraints

def get_realtime_weather(airport_icao: str) -> str:
    print(f"\n[TOOL CALLED: Fetching weather for {airport_icao}]")
    weather_data = {
        "KJFK": "TAF KJFK 051730Z 0518/0618 20008KT P6SM BKN030 OVC080 TEMPO 0600/0604 4SM SHRA BKN015 (Moderate risk of snow/low visibility in morning)",
        "ZSPD": "METAR ZSPD 051800Z 05005KT 8000 FEW030 BKN100 12/09 Q1015 NOSIG (Good VFR conditions expected)",
    }
    return weather_data.get(airport_icao.upper(), f"No real-time data found for {airport_icao}.")

def get_notam_data(region_icao: str) -> str:
    print(f"[TOOL CALLED: Fetching NOTAMs for {region_icao}]")
    notam_data = {
        "EAST_US": "NOTAM 05/045: Restricted Airspace R-400 active for military exercises 1400Z-1800Z. Alternate routing required.",
        "ASIA": "NOTAM 05/012: Due to volcanic activity, airway R-460 is permanently closed in Russian airspace. Alternate route requested via N-901.",
    }
    return notam_data.get(region_icao.upper(), f"No active NOTAMs of note for {region_icao}.")


# --- 4. AGENT EXECUTION (SINGLE-TURN LOGIC) ---

def run_edge_pilot_agent(
    initial_user_prompt: str,
    model_id: str = LFM2_MODEL_ID
) -> dict:

    if "YOUR_OPENROUTER_API_KEY_PLACEHOLDER" in OPENROUTER_API_KEY:
        return {"status": "error", "message": "API Key placeholder detected. Please replace 'YOUR_OPENROUTER_API_KEY_PLACEHOLDER' with your actual key."}

    client = OpenAI(
        base_url=OPENROUTER_BASE_URL,
        api_key=OPENROUTER_API_KEY,
        default_headers={"HTTP-Referer": "EdgePilotAgent-SingleTurn", "X-Title": "LFM2 B777 Planner"}
    )

    # --- STEP 1: EXECUTE LOCAL TOOLS AND GATHER DATA ---
    print("[AGENT STEP 1] Pre-fetching required external data (local simulation)...")
    weather_kjfk = get_realtime_weather("KJFK")
    weather_zspd = get_realtime_weather("ZSPD")
    notams_east = get_notam_data("EAST_US")
    notams_asia = get_notam_data("ASIA")

    context_data = f"""
    --- REAL-TIME FLIGHT CONTEXT DATA ---
    1. KJFK Weather (Departure): {weather_kjfk}
    2. ZSPD Weather (Arrival): {weather_zspd}
    3. East US NOTAMs (Departure Route): {notams_east}
    4. Asia/North Pole NOTAMs (En Route): {notams_asia}
    --- END CONTEXT ---
    """

    # --- STEP 2: BUILD THE SINGLE PROMPT ---
    messages = [
        {
            "role": "system",
            "content": (
                "You are EdgePilot, an expert Aviation Dispatch Agent. "
                "Your task is to generate the SAFEST and most OPTIMIZED flight plan for a Boeing 777 (long range) based on the user's request AND the provided CONTEXT DATA. "
                "**CRITICAL ROUTE CONSTRAINT: Since this is a long-haul international flight (KJFK to ZSPD), the 'route' array MUST contain a minimum of 10 waypoints to reflect realistic oceanic and transpolar tracks, including specific track names or oceanic entry/exit points.** " # <--- CRITICAL FIX: Explicit route count constraint
                "The final response MUST be a single JSON object that strictly conforms to the FlightPlanOutput Pydantic schema. "
                "Do not include any explanatory text or markdown formatting outside the single JSON block."
                # Using V2 Pydantic method
                f"SCHEMA: {json.dumps(FlightPlanOutput.model_json_schema())}"
            )
        },
        {"role": "user", "content": f"{initial_user_prompt}\n\n{context_data}"}
    ]

    # --- STEP 3: CALL LFM2 ---
    try:
        print("[AGENT STEP 2] Calling LFM2 Model for final plan generation (Single-Turn)...")
        response = client.chat.completions.create(
            model=model_id,
            messages=messages,
            temperature=0.0,
        )

        # --- STEP 4: PARSE AND VALIDATE OUTPUT ---
        raw_json = response.choices[0].message.content
        print("[AGENT SUCCESS] Final response received. Parsing JSON...")
        # Using V2 Pydantic methods
        return FlightPlanOutput.model_validate_json(raw_json).model_dump()

    except Exception as e:
        error_message = str(e)
        print(f"\n--- ERROR ---: {error_message}")
        return {"status": "error", "message": f"An unexpected error occurred: {error_message}"}

# --- 5. EXECUTION ---
if __name__ == "__main__":

    # New Example Request
    request = "Plan a flight from New York (KJFK) to Shanghai (ZSPD) for a Boeing 777, optimizing for the shortest possible route but factoring in all current NOTAMs and weather."

    print(f"REQUEST: {request}\n")

    # Run the agent
    final_plan = run_edge_pilot_agent(request)

    print("\n" + "="*50)
    print("✈️ FINAL LFM2 EDGE PILOT FLIGHT PLAN RESULT ✈️")
    print("="*50)
    print(json.dumps(final_plan, indent=4))
    print("\nAgentic Loop Complete.")

REQUEST: Plan a flight from New York (KJFK) to Shanghai (ZSPD) for a Boeing 777, optimizing for the shortest possible route but factoring in all current NOTAMs and weather.

[AGENT STEP 1] Pre-fetching required external data (local simulation)...

[TOOL CALLED: Fetching weather for KJFK]

[TOOL CALLED: Fetching weather for ZSPD]
[TOOL CALLED: Fetching NOTAMs for EAST_US]
[TOOL CALLED: Fetching NOTAMs for ASIA]
[AGENT STEP 2] Calling LFM2 Model for final plan generation (Single-Turn)...
[AGENT SUCCESS] Final response received. Parsing JSON...

✈️ FINAL LFM2 EDGE PILOT FLIGHT PLAN RESULT ✈️
{
    "flight_number": "B77W-110",
    "departure_icao": "KJFK",
    "arrival_icao": "ZSPD",
    "estimated_time_en_route_hrs": 14.5,
    "fuel_required_kg": 125000,
    "route": [
        {
            "waypoint": "KJFK",
            "type": "JET_WAY",
            "altitude_ft": 35000
        },
        {
            "waypoint": "DEP_APT_ATL",
            "type": "DEP_APT",
            "altitude_ft":